# Get stock info (HistPrice, Trend)

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
import time
import requests
import io
from yahoofinancials import YahooFinancials

In [2]:
# create list of S&P500 symbols
symbol = pd.read_csv("../04data/01stockprice/sp500symbol.csv")
symbol_list = symbol['Symbol'].tolist()

## Get HistPrice

In [14]:
# create empty dataframe
stock_final = pd.DataFrame()
start = datetime.datetime(2015,1,1)
end = datetime.datetime(2020,12,31)
# iterate over each symbol
for i in symbol_list:  
    
    # print the symbol which is being downloaded
    # print( str(symbol_list.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None


1 Failed download:
- BRK.B: No data found, symbol may be delisted

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted


In [15]:
stock_final.head()

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2014-12-31,28.205000,28.282499,27.552500,27.594999,25.137676,165613600.0,AAPL
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.898552,212818400.0,AAPL
2015-01-05,27.072500,27.162500,26.352501,26.562500,24.197117,257142000.0,AAPL
2015-01-06,26.635000,26.857500,26.157499,26.565001,24.199400,263188400.0,AAPL
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.538729,160423600.0,AAPL


In [18]:
## save hist price to csv
stock_final.to_csv("../04data/01stockprice/histprice.csv")

In [23]:
stock_final[stock_final['Name'] == 'AAP']

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2014-12-31,160.639999,162.690002,159.259995,159.279999,156.463760,468100.0,AAP
2015-01-02,160.850006,162.500000,157.470001,158.559998,155.756485,509800.0,AAP
2015-01-05,158.169998,158.990005,155.899994,156.470001,153.703476,803600.0,AAP
2015-01-06,156.559998,157.289993,153.899994,156.360001,153.595398,872300.0,AAP
2015-01-07,157.610001,160.610001,157.059998,159.720001,156.895981,1326800.0,AAP
...,...,...,...,...,...,...,...
2020-12-18,159.970001,161.039993,158.759995,159.990005,159.990005,1478300.0,AAP
2020-12-21,158.440002,160.580002,156.690002,160.490005,160.490005,1075200.0,AAP
2020-12-22,160.419998,161.179993,158.679993,159.399994,159.399994,780400.0,AAP


In [26]:
print("number of unique stock",len(stock_final['Name'].value_counts()))

number of unique stock 503


## Import Google trend data

In [4]:
from time import sleep

sleep(4)
print('Test sleep')

Test sleep


In [36]:
test = yf.Ticker("MA")
print(test.info['industry'])

Credit Services


In [3]:
# Import TrendReq and Setting
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360)

In [7]:
# test create code for getting monthly Google Trend Data Overtime for a single stock
kw_list = ['TSLA']
pytrends.build_payload(kw_list, cat=0, timeframe= 'all', geo='US',gprop='')
Tempdata = pd.DataFrame(pytrends.interest_over_time())
Tempdata = Tempdata.drop(columns= ['isPartial'])
print(kw_list)
print(Tempdata.tail(5))

['TSLA']
            TSLA
date            
2020-08-01    65
2020-09-01   100
2020-10-01    48
2020-11-01    49
2020-12-01    75


In [10]:
## Monthly Google Trend Data OverTime

## Create a blank dataframe with date as index for merging with data from TempData table later.

kw_list = ['NONE']
pytrends.build_payload(kw_list, cat=0, timeframe='all', geo='US', gprop='')  #'today 5-y'
TempData = pd.DataFrame(pytrends.interest_over_time())
GGTrendAllTimeMonthly = TempData.drop(columns=['isPartial','NONE'])



## Getting 'Monthly Google Trend Data OverTime' for the first 50 stocks in StockName
count = 0

for Stock in symbol_list :
   kw_list = [Stock]
   pytrends.build_payload(kw_list, cat=0, timeframe='all', geo='US', gprop='')  #'today 5-y'
   TempData = pd.DataFrame(pytrends.interest_over_time())
   TempData = TempData.drop(columns=['isPartial'])
   GGTrendAllTimeMonthly = pd.concat([GGTrendAllTimeMonthly, TempData], axis=1, join='inner')

   count += 1
   if count % 40 == 0:
      sleep(20)
#    print(kw_list)
   #print(GGTrendAllTimeMonthly.head(2))
   #print('------------------')

In [13]:
GGTrendAllTimeMonthly.describe()

,AAPL,MSFT,AMZN,FB,GOOGL,GOOG,TSLA,BRK.B,JNJ,JPM,...,UNM,FOX,GPS,SLG,FTI,XRX,HFC,UAA,UA,NWS
count,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,...,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000,204.000000
mean,38.446078,22.877451,17.598039,13.975490,42.112745,37.990196,8.975490,21.485294,32.323529,29.480392,...,63.823529,49.333333,37.313725,35.808824,51.318627,33.647059,42.666667,51.200980,55.980392,57.813725
std,18.267364,15.216328,21.689824,19.176504,23.216947,16.840308,15.023279,14.905772,13.156251,15.596027,...,15.097654,9.214823,14.678143,17.327836,15.952869,15.518397,19.089023,17.758195,22.395012,11.469154
min,1.000000,7.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,7.000000,4.000000,...,36.000000,32.000000,16.000000,0.000000,16.000000,0.000000,19.000000,17.000000,23.000000,32.000000
25%,27.000000,12.000000,4.000000,0.000000,24.750000,23.000000,0.000000,12.000000,24.000000,20.750000,...,50.750000,45.000000,26.000000,25.000000,40.750000,25.000000,28.000000,37.000000,37.000000,49.000000
50%,36.000000,17.000000,7.000000,10.000000,38.000000,38.000000,1.000000,18.000000,30.000000,26.000000,...,63.000000,49.000000,33.000000,32.000000,46.000000,32.000000,36.000000,49.000000,50.500000,58.000000
75%,49.000000,30.000000,22.000000,17.000000,60.500000,51.000000,12.000000,26.250000,38.250000,35.000000,...,76.000000,54.000000,47.000000,41.250000,59.000000,40.000000,55.250000,64.000000,80.250000,65.000000
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
